<a href="https://colab.research.google.com/github/strongeryoung/class_SmartDataAnalysis/blob/main/Text_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install konlpy
from gensim.models import Word2Vec
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import datetime
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.6 MB/s eta 0:00:00


In [ ]:
def check_len(str_1, len_num):
    if len(str_1) > len_num:
        return True
    return False


def make_noun_doc(full_text):
   # 문장 .를 기준으로 나눈다.
    result_list = []
    text_list = full_text.split('.')

    for text in text_list:
        twitter = Okt()
        text_nouns = twitter.nouns(text)
        #text_adj = twitter.adjective(text)
        result_list.append(text_nouns)

    return result_list


def make_all_doc(full_text):
    okt=Okt()
    #full_text.split('.')
    result = okt.pos(full_text)
    target_list = []
    for w,p in result:
         if p == "Verb" or p == "Noun" or p == 'Advjective' or p == 'Conjunction' or p == 'Josa':                    # 형태소 분석 결과 중에서 명사, 동사, 형용사를 분리하여 추출하도록 한다
                target_list.append(w)
    all_count = Counter(target_list)
    str_result = ' '.join(target_list)
    return  [target_list]

def sentiment_analyst(df, len_num):

    df[0] = df['text']
    df[0] = df[0].drop_duplicates()
    df = df.dropna()
    df[0] = df[0].apply(lambda x: re.sub('[^가-힣0-9a-zA-Z\\s]', '', x))
    df[0] = df[0].apply(lambda x: ' '.join(x.split()))

    df['remove_word'] = df[0].apply(lambda x: remove_words(x))
    df = df[df['remove_word'] == True]

    #print(df)
    df['len_check'] = df[0].apply(lambda x: check_len(x, len_num))
    df = df[df['len_check'] == True]
    txt_list = df[0].values.tolist()

     # str, int , float, list, dict
    noun_list = []
    sentence = []
    noun_cnt = []

  #print(df)

    for i in range(len(txt_list)):

        text = txt_list[i]
        nouns, str_, noun_count = make_noun_doc(text)
        noun_list.append(nouns)
        sentence.append(str_)
        noun_cnt.append(noun_count)

    noun_count_df = (pd.DataFrame(noun_cnt)).T.fillna(0)
    noun_count_df['freq'] = noun_count_df.sum(axis=1)
    return sentence


def get_senti_score(text, dictionary):
    text_nouns, str_result, noun_count = make_noun_doc(text)
    print(text_nouns)
    sent_df = dictionary
    try:
        tmp_df = sent_df.loc[text_nouns] # text_nouns는 index리스트!
        tmp_df= tmp_df.dropna()
        print(tmp_df)
        result =  tmp_df[['pos', 'neg','neu_score','pos_score','neg_score']].sum().values.tolist()
        print(result)
    except:
        result = [0.0,0.0,0.0]
    return result

def read_data(path, num):

    df_list = []

    for i in range(num):
        df = pd.read_excel(path.format(i))
        temp_df = df.drop_duplicates(0)
        df_list.append(temp_df)
    return df_list

def words_mean(sentence, vec_size):
    #model = Word2Vec.load('gdrive/MyDrive/비정형데이터분석_채권/DATA/word2vec_100_4_20210418.model')
# sentence의 형태는 명사로 이루어진 한 문장 (str), vec_size는 내가 학습시킨 word2vec에서의 vec사이즈
#   words = sentence.split()                                                    ## 입력된 문장을 space 기준으로 분리시킨다, 여기서는 [단어,단어,단어] 형태로 전환되었기 때문에 ''기준으로 분리할 필요가 없다
    vec_sum = np.zeros(vec_size, dtype=float)                                   ## 사전에 없는 경우를 대비하여 0 행렬을 형성한다
  # word2vec voca set
    word_vectors = model.wv                                                     ##  W2V이 학습한 단어의  set 이다. (일종의 사전역활을 한다)

    count = 0

    for word in sentence:
        if word in word_vectors:                                                ## 학습된 사전안에서 존재하는지 여부를 판단한다.
            count += 1
            vec_sum = np.add(vec_sum, model.wv[word])                           ## 1차원 행렬이다 !!, 사전에서 단어의 존재여부를 확인하여 존재 할경우 벡터값을 입력해준다. 없으면 넘어간다

    if count > 0:                                                               ## count를 측정하여 평균을 계산한다.
        vec_mean = vec_sum/count
    else:
        vec_mean = vec_sum                                                      ## 만약 count가 0이면 vec_mean = vec_sum 이다

    return vec_mean.tolist()

# [[],[],[]]
def concat_list(news):                                                          ## [[문장:단어,단어,단어], [문장:단어,단어,단어], [문장:단어,단어,단어]] -->기사 형태의 리스트 구조를 합해서 [단어,단어,단어,단어,단어,단어] --> 기사 형태로 전환시킨다
    result_list = []
    for sen in news:
        result_list = result_list + sen
    return result_list

def remove_words(words_list, stop_words):
    new_words_list = []
    word_list = words_list[0]
    for word in word_list:
        if word not in stop_words:
            new_words_list.append(word)
    return [new_words_list]

def get_text_date(start_year, end_year, last_month):
    date_list = []
    range_year = end_year + 1
    for y in range(start_year, range_year):
        if y != end_year:
            for m in range(1,13):
                dates = str(y) + '-' + str(m)
                date_list.append(dates)
        else:
            for m in range(1,last_month):
                dates = str(y) + '-' + str(m)
                date_list.append(dates)
    return date_list

In [ ]:
#path = 'gdrive/My Drive/비정형데이터분석_채권/DATA/학습용_문장종합_20200429.xlsx' (구버전으로 분류성능은 XGB로 74% )
## final_text_ver3는 이데일리 검색어를 "금리"로 확장하여 기사를 수집한것

path0 = '/content/drive/MyDrive/Smart_Data_Analysis_Class_202302_비공개/Week_12/sample_text.xlsx'

text_df1 = pd.read_excel(path0)

text_df = text_df1

stop_words = stop_words = ['광고', '신문']

text_df = text_df.dropna(axis=0)
text_df


,date,news_text
0,2023-11-21,[한라일보] 제주도는 누가 뭐래도 우리나라 최고의 관광지이다. 누구나 가서 머물고 ...
1,2023-11-20,한 주 내내 뺨이 베일 것 같은 칼바람이 불었습니다. 아직 마음으로 가을을 채 다 ...
2,2023-11-19,제주도 여행 비용이 해외여행의 반값이라도 가지 않는다는 사람들이 늘고 있다는 리서치...
3,2023-10-26,가을 여행철을 맞아 한창 성수기여야 할 제주가 ‘고물가 논란’ 등에 휩싸이면서 제대...
4,2023-10-11,‘화산섬 제주’의 신비로움을 걸으면서 체험하는 행사가 열린다. 제주도 세계유산본부(...
5,2023-09-29,"한국을 찾는 중국인 단체관광객 ‘유커’가 늘면서 면세점과 주요 상권, 관광지가 7년..."
6,2023-09-22,“최근 한 달 사이에 중국인 환자가 50% 가량 늘어난거 같아요. 추석 연휴에도 중...
7,2023-09-06,엔데믹(endemic·풍토병화된 감염병) 이후 해외여행 인구의 급증으로 여름 성수기...
8,2023-08-30,​제주연구원의 ‘후쿠시마 원전 오염수 방류 후 제주지역 관광 소비 의향’ 조사 결과...
9,2023-08-30,지난달 31일 오후 2시 제주시 제주항 국제크루즈터미널. 중국 상하이(上海)에서 출...


In [ ]:
text_df['year'] = text_df['date'].dt.year
text_df['month'] = text_df['date'].dt.month

## text 정재 3단계 과정을 거친다
text_df['words'] = text_df['news_text'].apply(lambda x:re.sub('[^A-Za-z가-힣\\s\.]', ' ', str(x)))
text_df['words'] = text_df['words'].apply(lambda x : re.sub('\n', ' ', x))
text_df['words'] = text_df['words'].apply(lambda x : ' '.join(x.split()))

In [ ]:
text_df['elements'] = text_df['news_text'].apply(lambda x : make_all_doc(x))        ## 명사, 동사, 형용사를 분리하는 작업을 수행하도록 한다
text_df['elements_clean'] = text_df['elements'].apply(lambda x : remove_words(x, stop_words))
text_df['news_nouns'] = text_df['elements_clean'].apply(lambda x : concat_list(x))

vec_size = 300
win_num = 3
model = Word2Vec.load('/content/drive/MyDrive/Smart_Data_Analysis_Class_202302_비공개/Week_12/word_model/word2vec_300_3_2023-11-22.model')
text_df['vec_mean'] = text_df['news_nouns'].apply(lambda x: words_mean(x, vec_size))


In [ ]:
text_df

,date,news_text,year,month,words,elements,elements_clean,news_nouns,vec_mean
0,2023-11-21,[한라일보] 제주도는 누가 뭐래도 우리나라 최고의 관광지이다. 누구나 가서 머물고 ...,2023,11,한라일보 제주도는 누가 뭐래도 우리나라 최고의 관광지이다. 누구나 가서 머물고 싶어...,"[[한라일보, 제주도, 는, 누가, 뭐, 래, 도, 우리나라, 최고, 의, 관광지,...","[[한라일보, 제주도, 는, 누가, 뭐, 래, 도, 우리나라, 최고, 의, 관광지,...","[한라일보, 제주도, 는, 누가, 뭐, 래, 도, 우리나라, 최고, 의, 관광지, ...","[-0.007234264654433452, 0.0838124800043053, -0..."
1,2023-11-20,한 주 내내 뺨이 베일 것 같은 칼바람이 불었습니다. 아직 마음으로 가을을 채 다 ...,2023,11,한 주 내내 뺨이 베일 것 같은 칼바람이 불었습니다. 아직 마음으로 가을을 채 다 ...,"[[한, 주, 내내, 뺨, 이, 베일, 것, 칼, 바람, 이, 불었습니다, 마음, ...","[[한, 주, 내내, 뺨, 이, 베일, 것, 칼, 바람, 이, 불었습니다, 마음, ...","[한, 주, 내내, 뺨, 이, 베일, 것, 칼, 바람, 이, 불었습니다, 마음, 으...","[-0.007157370916683522, 0.08019529849370628, -..."
2,2023-11-19,제주도 여행 비용이 해외여행의 반값이라도 가지 않는다는 사람들이 늘고 있다는 리서치...,2023,11,제주도 여행 비용이 해외여행의 반값이라도 가지 않는다는 사람들이 늘고 있다는 리서치...,"[[제주도, 여행, 비용, 이, 해외여행, 의, 반값, 이라도, 가지, 않는다는, ...","[[제주도, 여행, 비용, 이, 해외여행, 의, 반값, 이라도, 가지, 않는다는, ...","[제주도, 여행, 비용, 이, 해외여행, 의, 반값, 이라도, 가지, 않는다는, 사...","[-0.0069022693893485915, 0.07130504978932532, ..."
3,2023-10-26,가을 여행철을 맞아 한창 성수기여야 할 제주가 ‘고물가 논란’ 등에 휩싸이면서 제대...,2023,10,가을 여행철을 맞아 한창 성수기여야 할 제주가 고물가 논란 등에 휩싸이면서 제대로 ...,"[[가을, 여행, 철, 을, 맞아, 한창, 여야, 할, 제주, 가, 고물, 가, 논...","[[가을, 여행, 철, 을, 맞아, 한창, 여야, 할, 제주, 가, 고물, 가, 논...","[가을, 여행, 철, 을, 맞아, 한창, 여야, 할, 제주, 가, 고물, 가, 논란...","[-0.007292752531235727, 0.07821206495432656, -..."
4,2023-10-11,‘화산섬 제주’의 신비로움을 걸으면서 체험하는 행사가 열린다. 제주도 세계유산본부(...,2023,10,화산섬 제주 의 신비로움을 걸으면서 체험하는 행사가 열린다. 제주도 세계유산본부 본...,"[[화산섬, 제주, 의, 신비, 로움, 을, 걸으면서, 체험, 하는, 행사, 가, ...","[[화산섬, 제주, 의, 신비, 로움, 을, 걸으면서, 체험, 하는, 행사, 가, ...","[화산섬, 제주, 의, 신비, 로움, 을, 걸으면서, 체험, 하는, 행사, 가, 린...","[-0.006532786996387302, 0.07758084438994826, -..."
5,2023-09-29,"한국을 찾는 중국인 단체관광객 ‘유커’가 늘면서 면세점과 주요 상권, 관광지가 7년...",2023,9,한국을 찾는 중국인 단체관광객 유커 가 늘면서 면세점과 주요 상권 관광지가 년 만에...,"[[한국, 을, 찾는, 중국인, 단체, 관광객, 유, 커, 가, 늘면서, 면세점, ...","[[한국, 을, 찾는, 중국인, 단체, 관광객, 유, 커, 가, 늘면서, 면세점, ...","[한국, 을, 찾는, 중국인, 단체, 관광객, 유, 커, 가, 늘면서, 면세점, 과...","[-0.006660204510237612, 0.07547926651956192, -..."
6,2023-09-22,“최근 한 달 사이에 중국인 환자가 50% 가량 늘어난거 같아요. 추석 연휴에도 중...,2023,9,최근 한 달 사이에 중국인 환자가 가량 늘어난거 같아요. 추석 연휴에도 중국 관광객...,"[[최근, 한, 달, 사이, 에, 중국인, 환자, 가, 가량, 늘어난거, 추석, 연...","[[최근, 한, 달, 사이, 에, 중국인, 환자, 가, 가량, 늘어난거, 추석, 연...","[최근, 한, 달, 사이, 에, 중국인, 환자, 가, 가량, 늘어난거, 추석, 연휴...","[-0.006887645118706209, 0.07885379309115687, -..."
7,2023-09-06,엔데믹(endemic·풍토병화된 감염병) 이후 해외여행 인구의 급증으로 여름 성수기...,2023,9,엔데믹 endemic 풍토병화된 감염병 이후 해외여행 인구의 급증으로 여름 성수기 ...,"[[엔데믹, 풍토병, 된, 감염병, 이후, 해외여행, 인구, 의, 급증, 으로, 여...","[[엔데믹, 풍토병, 된, 감염병, 이후, 해외여행, 인구, 의, 급증, 으로, 여...","[엔데믹, 풍토병, 된, 감염병, 이후, 해외여행, 인구, 의, 급증, 으로, 여름...","[-0.006996966358049879, 0.07823608771425261, -..."
8,2023-08-30,​제주연구원의 ‘후쿠시마 원전 오염수 방류 후 제주지역 관광 소비 의향’ 조사 결과...,2023,8,제주연구원의 후쿠시마 원전 오염수 방류 후 제주지역 관광 소비 의향 조사 결과 응답...,"[[연구원, 의, 후쿠시마, 원전, 오염수, 방류, 후, 제주, 지역, 관광, 소비...","[[연구원, 의, 후쿠시마, 원전, 오염수, 방류, 후, 제주, 지역, 관광, 소비...","[연구원, 의, 후쿠시마, 원전, 오염수, 방류, 후, 제주, 지역, 관광, 소비,...","[-0.007642327572515785, 0.08317301609148245, -..."
9,2023-08-30,지난달 31일 오후 2시 제주시 제주항 국제크루즈터미널. 중국 상하이(上海)에서 출...,2023,8,지난달 일 오후 시 제주시 제주항 국제크루즈터미널. 중국 상하이 에서 출발한 크루즈...,"[[지난달, 오후, 제주시, 제주, 항, 국제, 크루즈, 터미널, 중국, 상하이, ...","[[지난달, 오후, 제주시, 제주, 항, 국제, 크루즈, 터미널, 중국, 상하이, ...","[지난달, 오후, 제주시, 제주, 항, 국제, 크루즈, 터미널, 중국, 상하이, 에...","[-0.0075696529641448304, 0.08258824696321762, ..."


In [ ]:
## XGB 학습데이터 사용하여 확률점수 부여

from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from joblib import dump, load
from sklearn.ensemble import AdaBoostClassifier

clf = load('/content/drive/MyDrive/Smart_Data_Analysis_Class_202302_비공개/Week_12/word_model/ada_model_300_3_2023-11-22.pkl')
X = pd.DataFrame(text_df['vec_mean'].values.tolist())

result = clf.predict(X)
result_prob = clf.predict_proba(X)
result_prob = pd.DataFrame(result_prob)
result = pd.Series(result)

# result_df = pd.concat([result_prob, result, text_df_sorted2[['year', 'month']]], axis=1)
# result_df.columns = ['negative', 'positive', 'result','year', 'month']

In [ ]:
result_prob

,0,1
0,0.500576,0.499424
1,0.500351,0.499649
2,0.501318,0.498682
3,0.499255,0.500745
4,0.499216,0.500784
5,0.498635,0.501365
6,0.498199,0.501801
7,0.498926,0.501074
8,0.498667,0.501333
9,0.499144,0.500856
